In [ ]:
#default_exp io.psm_reader.pfind_reader

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
import pandas as pd
import numpy as np

import alphabase.constants.modification as ap_mod

from alphabase.io.psm_reader.psm_reader import (
    PSMReaderBase, psm_reader_provider,
)

def convert_one_pFind_mod(mod):
    if mod[-1] == ')':
        mod = mod[:(mod.find('(')-1)]
        idx = mod.rfind('[')
        name = mod[:idx]
        site = mod[(idx+1):]
    else:
        idx = mod.rfind('[')
        name = mod[:idx]
        site = mod[(idx+1):-1]
    if len(site) == 1:
        return name + '@' + site
    elif site == 'AnyN-term':
        return name + '@' + 'Any N-term'
    elif site == 'ProteinN-term':
        return name + '@' + 'Protein N-term'
    elif site.startswith('AnyN-term'):
        return name + '@' + site[-1] + '^Any N-term'
    elif site.startswith('ProteinN-term'):
        return name + '@' + site[-1] + '^Protein N-term'
    elif site == 'AnyC-term':
        return name + '@' + 'Any C-term'
    elif site == 'ProteinC-term':
        return name + '@' + 'Protein C-term'
    elif site.startswith('AnyC-term'):
        return name + '@' + site[-1] + '^Any C-term'
    elif site.startswith('ProteinC-term'):
        return name + '@' + site[-1] + '^Protein C-term'
    else:
        return None

def translate_pFind_mod(mod_str):
    if not mod_str: return ""
    ret_mods = []
    for mod in mod_str.split(';'):
        mod = convert_one_pFind_mod(mod)
        if not mod: return pd.NA
        elif mod not in ap_mod.MOD_INFO_DICT: return pd.NA
        else: ret_mods.append(mod)
    return ';'.join(ret_mods)

def get_pFind_mods(pfind_mod_str):
    pfind_mod_str = pfind_mod_str.strip(';')
    if not pfind_mod_str: return "", ""

    items = [item.split(',',3) for item in pfind_mod_str.split(';')]
    items = list(zip(*items))
    return ';'.join(items[1]), ';'.join(items[0])

def remove_pFind_decoy_protein(protein):
    proteins = protein[:-1].split('/')
    return ';'.join(
        [
            protein for protein in proteins 
            if not protein.startswith('REV_')
        ]
    )


In [ ]:
#export
class pFindReader(PSMReaderBase):
    def __init__(self,
        *,
        column_mapping:dict = None,
        modification_mapping:dict = None,
        fdr = 0.01,
        keep_decoy = False,
    ):
        super().__init__(
            column_mapping=column_mapping,
            modification_mapping=modification_mapping,
            fdr = fdr,
            keep_decoy = keep_decoy,
        )

    def _init_column_mapping(self):
        self.column_mapping = {
            'sequence': 'Sequence',
            'charge': 'Charge',
            'raw_name': 'raw_name',
            'query_id': 'File_Name',
            'spec_idx': 'Scan_No',
            'score': 'Final_Score',
            'proteins': 'Proteins',
            'uniprot_ids': 'Proteins',
            'fdr': 'Q-value',
            'decoy': 'decoy'
        }
    def _init_modification_mapping(self):
        self.modification_mapping = {}

    def _translate_modifications(self):
        pass

    def _load_file(self, filename):
        pfind_df = pd.read_csv(filename, index_col=False, sep='\t')
        pfind_df.fillna('', inplace=True)
        pfind_df = pfind_df[pfind_df.Sequence != '']
        pfind_df['raw_name'] = pfind_df[
            'File_Name'
        ].str.split('.').apply(lambda x: x[0])
        # pfind_df['Proteins'] = pfind_df[
        #     'Proteins'
        # ].apply(remove_pFind_decoy_protein)
        pfind_df['decoy'] = (
            pfind_df['Target/Decoy']=='decoy'
        ).astype(np.int8)
        return pfind_df

    def _load_modifications(self, pfind_df):
        (
            self._psm_df['mods'], self._psm_df['mod_sites']
        ) = zip(*pfind_df['Modification'].apply(get_pFind_mods))

        self._psm_df['mods'] = self._psm_df['mods'].apply(
            translate_pFind_mod
        )
        
psm_reader_provider.register_reader('pfind', pFindReader)

In [ ]:
#hide
from io import StringIO
txt = StringIO("""File_Name	Scan_No	Exp.MH+	Charge	Q-value	Sequence	Calc.MH+	Mass_Shift(Exp.-Calc.)	Raw_Score	Final_Score	Modification	Specificity	Proteins	Positions	Label	Target/Decoy	Miss.Clv.Sites	Avg.Frag.Mass.Shift	Others
Ecoli-1to1to1-un-C13-N15-10mM-20150823.30507.30507.2.0.dta	30507	2074.030369	2	0	AMIEAGAAAVHFEDQLASVK	2074.027271	0.003098	35.299588	5.15726e-013	2,Oxidation[M];	3	gi|16131841|ref|NP_418439.1|/	173,K,K/	1|0|	target	0	0.948977	131070	0	0	0	262143	0	0	0	32
Ecoli-1to1to1-un-C13-N15-10mM-20150823.21592.21592.2.0.dta	21592	1901.970273	2	0	VVIVGCGAQGLNQGLNMR	1901.968323	0.001950	32.774898	3.37982e-012	6,Carbamidomethyl[C];17,Oxidation[M];	3	gi|16131632|ref|NP_418222.1|/	39,K,D/	1|0|0|	target	0	-0.762750	2046	0	0	0	131070	0	0	0	32
Ecoli-1to1to1-un-C13-N15-1000mM-20150823.48516.48516.2.0.dta	48516	2478.354259	2	0	TEIIAETGAGQHGVASALASALLGLK	2478.356108	-0.001849	35.699390	3.98494e-012		3	gi|16129222|ref|NP_415777.1|/	103,K,C/	1|	target	0	1.615154	7340030	0	0	0	16777214	0	0	0	4
Ecoli-1to1to1-un-C13-N15-1000mM-20150823.41206.41206.2.0.dta	41206	2863.379214	2	0	VHVSISNEGADTYLFGPGIDDSVDLSR	2863.374327	0.004887	33.596154	4.45008e-012		3	gi|16129141|ref|NP_415696.1|/	61,K,Y/	1|	target	0	0.476076	1896446	0	0	0	33554431	0	0	0	0
Ecoli-1to1to1-un-C13-N15-10mM-20150823.40178.40178.2.0.dta	40178	3005.455608	2	0	GMNTAVGDEGGYAPNLGSNAEALAVIAEAVK	3005.451916	0.003692	27.724815	9.86133e-012	2,Oxidation[M];	3	gi|16130686|ref|NP_417259.1|/	200,K,A/	1|0|	target	0	1.555942	40956	0	0	0	536870896	0	0	0	36
Ecoli-1to1to1-un-C13-N15-60mM-20150823.14730.14730.2.0.dta	14730	2262.060813	2	0	APGSSSSGANGDGSLAQSQTGAVVR	2262.059164	0.001649	29.205909	6.42718e-011	10,Deamidated[N];	3	CON_LYSC_LYSEN/	81,R,A/	1|0|	target	0	0.369568	327678	0	0	0	16777215	0	0	0	32
Ecoli-1to1to1-un-C13-N15-10mM-20150823.37020.37020.2.0.dta	37020	1825.014946	2	0	QVTIAQLEDVKPLLMK	1825.013859	0.001087	26.354755	9.42957e-010	0,Gln->pyro-Glu[AnyN-termQ];15,Oxidation[M];	3	gi|90111346|ref|NP_416371.4|/	100,K,S/	1|0|0|	target	1	-1.823228	16894	0	0	0	32766	0	0	0	32
Ecoli-1to1to1-un-C13-N15-150mM-20150823.41501.41501.3.0.dta	41501	2712.197421	3	0	EGDNYVVLSDILGDEDHLGDMDFK	2712.198013	-0.000592	27.073978	9.82619e-010	21,Unknown[M];	3	gi|145698316|ref|NP_417633.4|/	470,K,V/	1|0|	target	0	0.814438	65596	0	0	0	4194288	0	0	0	36
""")
pf_reader = psm_reader_provider.get_reader('pfind')
pf_reader.import_file(txt)
pf_reader.psm_df

/Users/zengwenfeng/opt/anaconda3/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


,sequence,charge,raw_name,query_id,spec_idx,score,proteins,uniprot_ids,fdr,decoy,nAA,mods,mod_sites,precursor_mz
0,QVTIAQLEDVKPLLMK,2,Ecoli-1to1to1-un-C13-N15-10mM-20150823,Ecoli-1to1to1-un-C13-N15-10mM-20150823.37020.3...,37020,9.429570e-10,gi|90111346|ref|NP_416371.4|/,gi|90111346|ref|NP_416371.4|/,0,0,16,Gln->pyro-Glu@Q^Any N-term;Oxidation@M,0;15,913.010613
1,VVIVGCGAQGLNQGLNMR,2,Ecoli-1to1to1-un-C13-N15-10mM-20150823,Ecoli-1to1to1-un-C13-N15-10mM-20150823.21592.2...,21592,3.379820e-12,gi|16131632|ref|NP_418222.1|/,gi|16131632|ref|NP_418222.1|/,0,0,18,Carbamidomethyl@C;Oxidation@M,6;17,951.487845
2,AMIEAGAAAVHFEDQLASVK,2,Ecoli-1to1to1-un-C13-N15-10mM-20150823,Ecoli-1to1to1-un-C13-N15-10mM-20150823.30507.3...,30507,5.157260e-13,gi|16131841|ref|NP_418439.1|/,gi|16131841|ref|NP_418439.1|/,0,0,20,Oxidation@M,2,1037.517322
3,APGSSSSGANGDGSLAQSQTGAVVR,2,Ecoli-1to1to1-un-C13-N15-60mM-20150823,Ecoli-1to1to1-un-C13-N15-60mM-20150823.14730.1...,14730,6.427180e-11,CON_LYSC_LYSEN/,CON_LYSC_LYSEN/,0,0,25,Deamidated@N,10,1131.533274
4,TEIIAETGAGQHGVASALASALLGLK,2,Ecoli-1to1to1-un-C13-N15-1000mM-20150823,Ecoli-1to1to1-un-C13-N15-1000mM-20150823.48516...,48516,3.984940e-12,gi|16129222|ref|NP_415777.1|/,gi|16129222|ref|NP_415777.1|/,0,0,26,,,1239.681753
5,VHVSISNEGADTYLFGPGIDDSVDLSR,2,Ecoli-1to1to1-un-C13-N15-1000mM-20150823,Ecoli-1to1to1-un-C13-N15-1000mM-20150823.41206...,41206,4.450080e-12,gi|16129141|ref|NP_415696.1|/,gi|16129141|ref|NP_415696.1|/,0,0,27,,,1432.190867
6,GMNTAVGDEGGYAPNLGSNAEALAVIAEAVK,2,Ecoli-1to1to1-un-C13-N15-10mM-20150823,Ecoli-1to1to1-un-C13-N15-10mM-20150823.40178.4...,40178,9.861330e-12,gi|16130686|ref|NP_417259.1|/,gi|16130686|ref|NP_417259.1|/,0,0,31,Oxidation@M,2,1503.229666


In [ ]:
assert pf_reader.psm_df.mods.values[3] == 'Deamidated@N'
assert pf_reader.psm_df.mods.values[0] == 'Gln->pyro-Glu@Q^Any N-term;Oxidation@M'
assert pf_reader.psm_df.mod_sites.values[3] == '10'
assert pf_reader.psm_df.mod_sites.values[0] == '0;15'